In [1]:
# Parameters
INPUT_CSV = "C:/Users/Jason Pohl/OneDrive - Bond University/PhD/rff/NEW_DATA.csv"
OUTPUT_ROOT = "C:/Users/Jason Pohl/OneDrive - Bond University/PhD/rff/outputs_rff"
STAGE1_CFG = ""
SWAN_YEAR = 2008
WIN_START = 2004
WIN_END = 2012
RUN_TAG = "myUniqueRunId"


In [2]:
"""
STAGE 2 · RESILIENCE METRICS & BASELINE RATIOS
────────────────────────────────────────────────────────────────────────
Self-contained, works both as a stand-alone script and inside the same
Python session right after Stage 1.

Key features
• No manual parameters – everything comes from pipeline_config.yaml.
• Robust when __file__ is missing (e.g. Jupyter).
• Accepts either int or str keys under events: in the YAML.
• Re-uses `data_stage_1` if it exists in memory; otherwise loads the
  latest Stage 1 CSV.
• Produces  
  <OUTPUT_ROOT>/event=<SWAN_YEAR>/<RUN_DATE>/stage02/Stage2_Data_WithMetrics.csv
"""

from __future__ import annotations
from pathlib import Path
import os, sys, logging, yaml, io
from typing import Dict, List

import numpy as np
import pandas as pd

# ──────────────────────────────────────────────────────────────────────
# 0-3 · UNIVERSAL BOOTSTRAP  (cfg, run-path resolver, params, logger)
# ──────────────────────────────────────────────────────────────────────
from pipeline_utils import load_cfg, resolve_run_dir      # NEW shared helper
from pathlib import Path
import os, sys, logging, io, yaml
from typing import Dict, List

# 0 · read YAML once ---------------------------------------------------
CFG        = load_cfg()
defaults: Dict = CFG.get("defaults", {})
events:   Dict = CFG.get("events",   {})
st2_cfg:  Dict = CFG.get("stage2",   {})

# helper: treat YAML keys as both str and int
event_keys_str = {str(k): v for k, v in events.items()}

# 1 · core params -------------------------------------------------------
SWAN_YEAR = str(os.getenv("SWAN_YEAR") or next(iter(event_keys_str)))
if SWAN_YEAR not in event_keys_str:
    raise KeyError(f"SWAN_YEAR={SWAN_YEAR} not present in YAML `events:` block")

MAX_YEARS = int(st2_cfg.get("max_years", 4))
DATE_COL  = st2_cfg.get("date_col", "ReportDate")
ID_COL    = st2_cfg.get("id_col",   "Symbol")

METRICS: List[str] = st2_cfg.get(
    "metrics",
    [
        "NetIncome", "EarningBeforeInterestAndTax", "OperatingIncome", "EBITDA",
        "OperatingCashFlow", "FreeCashFlow", "Cash", "CashAndCashEquivalents",
        "TotalRevenue", "GrossProfit",
    ],
)
METRIC_SIGN: Dict[str, bool] = {m: True for m in METRICS}   # True ⇒ higher better

# 2 · resolve run folder & paths ---------------------------------------
#     • honour $RUN_DIR or $RUN_DATE if the user sets them
#     • otherwise pick the latest run that already contains Stage-1 output
RUN_DIR   = resolve_run_dir(must_have="stage01/stage01_cleaned.csv")
RUN_DATE  = RUN_DIR.name

STAGE1_FILE = RUN_DIR / "stage01" / "stage01_cleaned.csv"
OUT_DIR     = RUN_DIR / "stage02"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 3 · logger -----------------------------------------------------------
if "logger" in globals() and isinstance(globals()["logger"], logging.Logger):
    logger = logging.getLogger(__name__)
    if not any(isinstance(h, logging.FileHandler) and h.baseFilename.endswith("stage02.log")
               for h in logger.handlers):
        logger.addHandler(logging.FileHandler(OUT_DIR / "stage02.log", mode="w", encoding="utf-8"))
else:
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s | %(levelname)-7s | %(message)s",
        handlers=[
            logging.FileHandler(OUT_DIR / "stage02.log", mode="w", encoding="utf-8"),
            logging.StreamHandler(sys.stdout),
        ],
    )
    logger = logging.getLogger(__name__)

logger.info("==========  STAGE 2: RESILIENCE METRICS ==========")
logger.info("RUN_DIR        : %s", RUN_DIR)
logger.info("SWAN_YEAR=%s  RUN_DATE=%s  MAX_YEARS=%s", SWAN_YEAR, RUN_DATE, MAX_YEARS)
logger.info("DATE_COL / ID_COL = %s / %s", DATE_COL, ID_COL)
# ──────────────────────────────────────────────────────────────────────
# ──────────────────────────────────────────────────────────────────────
# 4 · LOAD STAGE 1 DATA  (memory → disk fallback)
# ──────────────────────────────────────────────────────────────────────
if "data_stage_1" in globals():
    df = globals()["data_stage_1"].copy()
    logger.info("Stage 1 data reused from memory.")
else:
    if not STAGE1_FILE.is_file():
        raise FileNotFoundError(f"Stage 1 CSV not found at {STAGE1_FILE}")
    df = pd.read_csv(STAGE1_FILE, parse_dates=[DATE_COL], low_memory=False)
    logger.info("Stage 1 CSV loaded: %s rows", f"{len(df):,}")

df["Year"] = df[DATE_COL].dt.year.astype("Int16")

# ──────────────────────────────────────────────────────────────────────
# 5 · DERIVED COLUMNS
# ──────────────────────────────────────────────────────────────────────
if {"PretaxIncome", "EffectiveTaxRateAsReported"}.issubset(df.columns):
    df["IncomeTaxExpense"] = df["PretaxIncome"] * df["EffectiveTaxRateAsReported"]
    med = df.groupby(["SectorName", "Year"])["IncomeTaxExpense"].transform("median")
    df["IncomeTaxExpense"].fillna(med, inplace=True)
else:
    df["IncomeTaxExpense"] = np.nan
    logger.warning("IncomeTaxExpense derivation skipped (missing inputs)")

if {"TotalAssets", "TotalLiabilitiesAsReported"}.issubset(df.columns):
    df["NetAssets"] = df["TotalAssets"] - df["TotalLiabilitiesAsReported"]
elif {"TotalAssets", "TotalLiabilities"}.issubset(df.columns):
    df["NetAssets"] = df["TotalAssets"] - df["TotalLiabilities"]
else:
    df["NetAssets"] = np.nan
    logger.warning("NetAssets derivation skipped (missing inputs)")

# ──────────────────────────────────────────────────────────────────────
# 6 · HELPERS
# ──────────────────────────────────────────────────────────────────────
SWAN_YEAR_INT = int(SWAN_YEAR)           # for numeric comparison

def _last_pre_swan(series: pd.Series) -> float:
    pre = series.dropna()
    pre = pre[pre.index < SWAN_YEAR_INT]
    return pre.iloc[-1] if not pre.empty else np.nan

def _first_recovery(series: pd.Series, baseline: float, higher_ok: bool) -> float:
    if pd.isna(baseline):
        return np.nan
    cond = series >= baseline if higher_ok else series <= baseline
    cand = series[(series.index >= SWAN_YEAR_INT) & cond]
    return cand.index.min() if not cand.empty else np.nan

# ──────────────────────────────────────────────────────────────────────
# 7 · METRIC LOOP
# ──────────────────────────────────────────────────────────────────────
for metric in METRICS:
    if metric not in df.columns:
        logger.warning("⏭️ %-30s missing — skipped", metric)
        continue

    grp = df.set_index("Year").groupby(ID_COL)[metric]

    baseline = grp.apply(_last_pre_swan).rename("Baseline")
    rec_year = grp.apply(
        lambda s: _first_recovery(s, baseline.get(s.name), METRIC_SIGN[metric])
    ).rename("RecYear")

    rp = (rec_year - SWAN_YEAR_INT + 1).clip(lower=1, upper=MAX_YEARS) \
                                       .fillna(MAX_YEARS).astype("int16")
    score = ((rp - 1) / (MAX_YEARS - 1)).round(4)     # 0 best … 1 worst
    flag  = (rp < rp.median()).astype("int8")

    df = (
        df.merge(rp.rename(f"RP_{metric}"),       on=ID_COL, how="left")
          .merge(score.rename(f"Score_{metric}"), on=ID_COL, how="left")
          .merge(flag.rename(f"Flag_{metric}"),   on=ID_COL, how="left")
    )

    logger.info("%-30s baseline %.1f%% | recovery %.1f%%",
                metric, baseline.notna().mean()*100, rp.notna().mean()*100)

# ──────────────────────────────────────────────────────────────────────
# 8 · EXPORT
# ──────────────────────────────────────────────────────────────────────
out_csv = OUT_DIR / "Stage2_Data_WithMetrics.csv"
df.to_csv(out_csv, index=False)

buf = io.StringIO(); df.info(buf=buf)
logger.info("Final DataFrame info:\n%s", buf.getvalue())
logger.info("Saved Stage 2 CSV → %s", out_csv)

data_stage_2 = df.copy()
logger.info("✅ STAGE 2 complete — `data_stage_2` ready")

2025-06-11 14:40:01,219 | INFO    | ==========  STAGE 2: RESILIENCE METRICS ==========


2025-06-11 14:40:01,221 | INFO    | RUN_DIR        : outputs_rff\daily\2025-06-11


2025-06-11 14:40:01,222 | INFO    | SWAN_YEAR=2008  RUN_DATE=2025-06-11  MAX_YEARS=4


2025-06-11 14:40:01,224 | INFO    | DATE_COL / ID_COL = ReportDate / Symbol


2025-06-11 14:40:02,305 | INFO    | Stage 1 CSV loaded: 34,862 rows


...

2025-06-11 14:40:13,655 | INFO    | TotalRevenue                   baseline 48.9% | recovery 100.0%


2025-06-11 14:40:14,927 | INFO    | GrossProfit                    baseline 48.9% | recovery 100.0%


2025-06-11 14:40:19,631 | INFO    | Final DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34862 entries, 0 to 34861
Columns: 196 entries, Symbol to Flag_GrossProfit
dtypes: Int16(1), datetime64[ns](1), float64(180), int64(4), object(10)
memory usage: 52.0+ MB



2025-06-11 14:40:19,632 | INFO    | Saved Stage 2 CSV → outputs_rff\daily\2025-06-11\stage02\Stage2_Data_WithMetrics.csv


2025-06-11 14:40:19,696 | INFO    | ✅ STAGE 2 complete — `data_stage_2` ready
